## Import Dependencies and Open Instrument Handles

In [1]:
import sys
import os
import time
import pyvisa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from binascii import unhexlify
import serial

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\Drivers")

from Driver_OSA_MS9740B import *
from Driver_TSL_Quantifi import *

In [2]:
rm = pyvisa.ResourceManager()

osa = OSA_MS9740B()
osa.open(rm)

tsl = TSL_Quantifi_1001_2_FC()
tsl.open(rm)

OSA Attenuation Status:  ON



## Initialization

In [11]:
wavelength_start = 1530
wavelength_stop = 1570
wavelength_points = 2001
wavelength_start_tsl = 1540
wavelength_stop_tsl = 1560
wavelength_points_tsl = 201
power = 8.0
OSA_Resolution = 0.03
OSA_VBW = 1000

## Function Definition

In [12]:
def sweepOSA(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW):
    osa.set_wavelength(wavelength_start, wavelength_stop, wavelength_points)
    osa.set_resolution_VBW(OSA_Resolution,OSA_VBW)
    osa.sweep_single()
    Power_list = osa.get_sweep_result()
    Power_list_uW = [None] * len(Power_list)
    for i in range(0,len(Power_list)):
        Power_list_uW[i] = 10**(float(Power_list[i])/10)*1000 
    Wavelength_list = np.linspace(wavelength_start,wavelength_stop,wavelength_points)
    Wavelength_peak_nm, Power_peak_dBm = osa.get_peak()   
    if Power_peak_dBm >= -30:
        SMSR_Linewidth_nm, SMSR_dB = osa.get_SMSR()
        Linewidth_3db_nm = osa.get_linewidth(3)
        Linewidth_20db_nm = osa.get_linewidth(20)
    else:
        SMSR_Linewidth_nm, SMSR_dB = 0, 0
        Linewidth_3db_nm = 0
        Linewidth_20db_nm = 0
    return  Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm

## Measurement Routine

In [18]:
meas_df = pd.DataFrame()
wavelength = np.linspace(wavelength_start_tsl,wavelength_stop_tsl,wavelength_points_tsl)
tsl.set_power(4,1,8)
t0 = time.time()
for k in range(0,wavelength_points_tsl,1):
    tsl.set_wavelength(4,1,wavelength[k])
    time.sleep(5)
    t_meas = time.time()-t0
    Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm = sweepOSA(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW)
    meas_df_temp = pd.DataFrame({'Time':t_meas, 'Wavelength_nm':Wavelength_list, 'Power_dBm':Power_list, 
                            'Power_uW':Power_list_uW, 'Wavelength_peak_nm':Wavelength_peak_nm, 'Power_peak_dBm':Power_peak_dBm,
                            'SMSR_dB':SMSR_dB, 'SMSR_linewidth_nm':SMSR_Linewidth_nm,
                            '3dB_linewidth_nm':Linewidth_3db_nm, '20dB_linewidth_nm':Linewidth_20db_nm})
    data = [meas_df_temp, meas_df]
    meas_df = pd.concat(data,ignore_index=True,sort=False)

In [20]:
meas_df

,Time,Wavelength_nm,Power_dBm,Power_uW,Wavelength_peak_nm,Power_peak_dBm,SMSR_dB,SMSR_linewidth_nm,3dB_linewidth_nm,20dB_linewidth_nm
0,1342.519326,1530.00,-56.44,2.269865e-03,1560.12,7.12,54.71,2.50,0.086,0.086
1,1342.519326,1530.02,-58.79,1.321296e-03,1560.12,7.12,54.71,2.50,0.086,0.086
2,1342.519326,1530.04,-69.4,1.148154e-04,1560.12,7.12,54.71,2.50,0.086,0.086
3,1342.519326,1530.06,-55.45,2.851018e-03,1560.12,7.12,54.71,2.50,0.086,0.086
4,1342.519326,1530.08,-60.13,9.705100e-04,1560.12,7.12,54.71,2.50,0.086,0.086
...,...,...,...,...,...,...,...,...,...,...
402196,5.139106,1569.92,-66.25,2.371374e-04,1540.10,7.03,55.76,2.42,0.038,0.038
402197,5.139106,1569.94,-55.7,2.691535e-03,1540.10,7.03,55.76,2.42,0.038,0.038
402198,5.139106,1569.96,-55.19,3.026913e-03,1540.10,7.03,55.76,2.42,0.038,0.038
402199,5.139106,1569.98,-120.0,1.000000e-09,1540.10,7.03,55.76,2.42,0.038,0.038


In [22]:
meas_df

,Time,Wavelength_nm,Power_dBm,Power_uW,Wavelength_peak_nm,Power_peak_dBm,SMSR_dB,SMSR_linewidth_nm,3dB_linewidth_nm,20dB_linewidth_nm
0,1342.519326,1530.00,-56.44,2.269865e-03,1560.12,7.12,54.71,2.50,0.086,0.086
1,1342.519326,1530.02,-58.79,1.321296e-03,1560.12,7.12,54.71,2.50,0.086,0.086
2,1342.519326,1530.04,-69.4,1.148154e-04,1560.12,7.12,54.71,2.50,0.086,0.086
3,1342.519326,1530.06,-55.45,2.851018e-03,1560.12,7.12,54.71,2.50,0.086,0.086
4,1342.519326,1530.08,-60.13,9.705100e-04,1560.12,7.12,54.71,2.50,0.086,0.086
...,...,...,...,...,...,...,...,...,...,...
402196,5.139106,1569.92,-66.25,2.371374e-04,1540.10,7.03,55.76,2.42,0.038,0.038
402197,5.139106,1569.94,-55.7,2.691535e-03,1540.10,7.03,55.76,2.42,0.038,0.038
402198,5.139106,1569.96,-55.19,3.026913e-03,1540.10,7.03,55.76,2.42,0.038,0.038
402199,5.139106,1569.98,-120.0,1.000000e-09,1540.10,7.03,55.76,2.42,0.038,0.038


## Save Data

In [23]:
meas_df.to_csv(r"TSL_Sweep.csv")

## Close Instrument Handles

In [21]:
osa.close()
tsl.close()